In [1]:
import requests
import time
import pandas as pd
import datetime as dt

In [2]:
base_api_url = 'https://api.socio.center/public/priority'
answer = requests.get(f"{base_api_url}/list")
answer_json = answer.json()

In [3]:
# здесь получаю данные по id и shortname универов
if answer.status_code == 200 and answer_json['status'] == 'success':
    university_names = pd.DataFrame(answer_json['data']['participants'])
    # карта применяется для перевода 1-5 в нужные значения в столбике
    university_names.group = university_names.group.map(answer_json['data']['group'])
    university_names = university_names.loc[university_names.level == '1']  # трек "Исследовательское лидерство"
else:
    raise Exception(f"{answer.status_code} не 200, либо {answer_json['status']} не success")

In [4]:
assert university_names.shape[0] == 15, "Число университетов не равно 15!"

In [5]:
combined_universities = pd.DataFrame()  # здесь суммарные данные

In [6]:
for index, row in university_names.iterrows():  # перебираю универы
    university_id, university_name = row['id'], row['shortName']
    university_url = f"{base_api_url}/{university_id}/indicators"
    university_answer = requests.get(university_url)
    university_json = university_answer.json()

    if university_answer.status_code == 200 and university_json['status'] == 'success':
        # здесь данные с базового (0), специального (1) треков, и финансовых показателей (2)
        indicators_data = university_json['data'][0]['elements'] + university_json['data'][1]['elements'] + university_json['data'][2]['elements']
        all_indicators_df = pd.DataFrame()  # это суммарная таблица по универу

        for indicator in indicators_data:
            # полное имя показателя вместе с описанием и единицами измерения
            indicator_name = f"{indicator['indicator']}. {indicator['description']} ({indicator['unit']})."
            current_indicator = pd.DataFrame()  # здесь будет лежать суммарная информация по каждому из показателей

            # в data лежат данные по самому показателю: факт и план, а в calculationData расчётные показатели
            for sub_indicator in indicator['data'] + indicator['calculationData']:
                # внутри лежит словарь с 10 связками ключ-значение по одному столбику, где напротив каждого года - его значение
                all_data = sub_indicator['data'].items()

                # здесь данные по каждому столбику по отдельности
                # делаю формат таким, чтобы на месте индекса был год, а единственный столбик - данные
                current_sub_indicator = pd.DataFrame(all_data).set_index(0).rename(
                    {1: f"{sub_indicator['description']}"}, axis=1)
                
                # в конце цикла прибавляю по одному столбику с данными
                # приклеиваю справа по одному
                current_indicator = pd.concat([current_indicator, current_sub_indicator], axis=1).fillna(0)

            # в конце получается табличка по показателю: факт, план и остальные, если они есть
            current_indicator = current_indicator.rename(
                {"Отражение факта": f"{indicator_name} Факт.",
                 'План': f"{indicator_name} План."}, axis=1)

            # здесь прибавляю табличку по данному показателю к общей табличке по университету
            all_indicators_df = pd.concat([all_indicators_df, current_indicator], axis=1)
    else:
        raise Exception(f'''По университету {university_name}
        {university_answer.status_code} не 200, либо {university_json['status']} не success''')

    # проставляю имя университета
    all_indicators_df['Университет'] = university_name
    # последнее действие по университету: суммарные данные по университету приклеиваю к общей таблице
    combined_universities = pd.concat([combined_universities, all_indicators_df])
    time.sleep(0.5)

In [7]:
# так как в цикле по индексу были годы, то обратно делаю столбик с годами
combined_universities['Год'] = combined_universities.index

In [8]:
# все колонки кроме 'Университет' приводим к числовому формату
for column_number in range(combined_universities.shape[1]):
    if combined_universities.iloc[:, column_number].name != 'Университет':
        combined_universities.iloc[:, column_number] = pd.to_numeric(combined_universities.iloc[:, column_number])

In [9]:
# сортировка и после сортировки создание столбика с №
combined_universities = combined_universities.sort_values(['Год', 'Университет'])
combined_universities['№'] = range(1, combined_universities.shape[0] + 1)

In [10]:
# конкатенация с правильным порядком столбиков
final_df = pd.concat([
    combined_universities.iloc[:, -1:],
    combined_universities.iloc[:, -3:-1],
    combined_universities.iloc[:, :-3]
], axis=1)

In [11]:
file_name = rf'C:/python_work/выгрузки/программа_2030/приоритет_2030_{dt.date.today()}.xlsx'
final_df.to_excel(file_name, sheet_name='Показатели', index=False)